# <center> Taller Keras $-$ Ejemplo 3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from time import time

from tensorflow.keras.models import Model, Sequential, load_model, save_model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical

from pandas import DataFrame
from sklearn.metrics import confusion_matrix, classification_report
from plotmatrix import pretty_plot_confusion_matrix, PlotMatrix

import warnings
warnings.filterwarnings('ignore')

# Introducción a las Redes Neuronales (NNs)

__Objetivos:__
* Entender el concepto de Dropout y su importancia en Deep Learning.
* Implementar una NN con Dropout en Keras para clasificar la base de datos MNIST.

In [ ]:
# cargar la interfaz a la base de datos que viene con Keras
from tensorflow.keras.datasets import mnist

# lectura de los datos
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print('Raw data shapes:')
print('X train:', train_images.shape, 'Y train:', train_labels.shape)
print('X test:', test_images.shape, 'Y test:', test_labels.shape)

# pre-procesamiento de los datos
train_images = train_images.reshape((60000, -1))
train_images = train_images.astype('float32') / 255.

test_images = test_images.reshape((10000, -1))
test_images = test_images.astype('float32') / 255.

# one-hot encoding
from tensorflow.keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels  = to_categorical(test_labels)

In [ ]:
print('Modified data shapes:')
print('X train:', train_images.shape, 'Y train:', train_labels.shape)
print('X test:', test_images.shape, 'Y test:', test_labels.shape)

data_shape = train_images.shape[1:]

print('')
print('data shape:', data_shape)

## Dropout

Dropout es una técnica que sirve para regularizar el entrenamiento de una red neuronal. Fue introducida por Srivastava, Hinton, Krizhevsky, Sutskever y Salakhutdinov en 2014.

Se utiliza en la práctica para reducir el sobre-entrenamiento (_overfitting_) de la red, es decir, reducir la diferencia entre el error de entrenamiento y el error de validación/prueba.

La técnica consiste en que, durante cada batch, se eliminan un cierto porcentaje de neuronas, seleccionadas aleatoriamente, de la capa donde se aplica el Dropout. Esto permite que el entrenamiento no depende fuertemente de neuronas específicas, y que los pesos se distribuyen más adecuadamente dentro de las neuronas que sobreviven al drop-out.

Al entrenar el siguiente batch, la totalidad de neuronas se repobla completamente y se hace una nueva selección aleatoria de neuronas a eliminar.

![](dropout.png)

A continuación definimeros una red neuronal de 4 capas ocultas, intercalando entre cada una, una capa de Dropout.

El parámetro en una capa Dropout es:

__rate:__ Porcentaje ($0 \leq rate < 1$) de neuronas a eliminar aleatoriamente.

Así, si se quieren eliminar el 40% de las neuronas, en Keras se agrega la capa

    layers.Dropout(rate=0.4, name='name')

__Ejercicio:__ En la siguiente red, adicione 4 capas Dropout, una después de cada capa densa oculta. Utilice una tasa de eliminación del 25% en cada una. (Asegúrese que los nombres de las capas sean distintos).

In [ ]:
### START CODE HERE ### (≈ 4 lines of code)
#network3 = models.Sequential()
#network3.add(layers.Dense(512, activation='relu', input_shape=data_shape, name='dense1'))

#network3.add(layers.Dense(512, activation='relu', name='dense2'))

#network3.add(layers.Dense(256, activation='relu', name='dense3'))

#network3.add(layers.Dense(128, activation='relu', name='dense4'))

#network3.add(layers.Dense(10, activation='softmax', name='classifier'))
### END CODE HERE ###



def NNmulticapaDropout(input_shape):
    I = Input(shape=input_shape, name='input')
    X = Dense(512, activation='relu', name='dense1')(I)
    X = Dropout(rate=0.25, name='drop1')(X)
    X = Dense(512, activation='relu', name='dense2')(X)
    X = Dropout(rate=0.25, name='drop2')(X)
    X = Dense(256, activation='relu', name='dense3')(X)
    X = Dropout(rate=0.25, name='drop3')(X)
    X = Dense(128, activation='relu', name='dense4')(X)
    X = Dropout(rate=0.25, name='drop4')(X)
    X = Dense(10, activation='softmax', name='classifier')(X)
    model = Model(I, X, name='NN-Multicapa-Dropout')
    return model

In [ ]:
if 'network3' in globals(): del network3
network3 = None

network3 = NNmulticapaDropout(data_shape)

In [ ]:
network3.summary()

### Compile

In [ ]:
alpha = 1e-4
opt = optimizers.RMSprop(lr=alpha)
network3.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

### Train

In [ ]:
tic = time()
history3 = network3.fit(train_images, train_labels, epochs=15, batch_size=64, validation_split=0.16666)
toc = time()
print('total training time:', toc-tic, 'seconds')

### Ver el historial de desempeño de la red

In [ ]:
history_dict = history3.history
history_dict.keys()

In [ ]:
acc = history3.history['accuracy']
val_acc = history3.history['val_accuracy']
loss = history3.history['loss']
val_loss = history3.history['val_loss']

epochs = range(1, len(acc)+1)

# figure
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(epochs, acc, 'ro', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

### Resultados

In [ ]:
# compute predictions (inference process)
predictions3 = network3.predict(test_images)

# from predictions compute most probable class
pred3 = np.argmax(predictions3, 1)
test_labs = np.argmax(test_labels, 1)

In [ ]:
print(classification_report(test_labs, pred3))

In [ ]:
CM3 = confusion_matrix(test_labs, pred3)

# plot confusion matrix
cf3 = DataFrame(CM3)
pretty_plot_confusion_matrix(cf3, annot=True, pred_val_axis='x', figsize=(10,10))

In [ ]:
# other plot
PlotMatrix(CM3, figsize=(10,10), cmap=plt.cm.Blues, title='Confusion Matrix')

---

---